# Sparse Word Representations Assignment (50 points)

**Objective:** Implement and apply sparse word representations using co-occurrence counts and TF-IDF weighting on a small document collection.

**Dataset:** We will use the 20 Newsgroups dataset, readily available in scikit-learn. We'll limit it to a subset of categories and a reduced number of documents to keep things manageable.

```
Instructions:
Make a copy of this code notebook into your own account.
Fill out the parts that are specified with
% -- Your Implementation -- %
Then submit your completed notebook with all the outputs to Gradescope.
```

### 1. Data Loading and Preprocessing (5 points)

Steps:
- Load the 20 Newsgroups dataset using scikit-learn, but only select 4 categories of your choice.
- Limit the dataset to the first 1000 documents.
- Tokenize the documents into words. You can use a simple tokenizer like splitting on spaces and remove any punctuations.
    - For this step you are welcome to use python's built in functions/libraries such as `re`, `str`, `string.punctuation`, etc. But you are not allowed to use more specialized functions such as `nltk` or `spacy`.
    - Regarding punctuation, there is some flexibility in this and you are welcome to make some design decisions where needed (e.g., whether to split on apostrophes). Please specify any design decisions or assumptions you make in your submission.
- Create a vocabulary of the 5000 most frequent words across all documents.

In [2]:
#
%pip install scikit-learn
from typing import List, Set, Dict, Tuple, Union
from collections import Counter
import numpy as np
from math import log
from sklearn.datasets import fetch_20newsgroups


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.datasets import fetch_20newsgroups

# Load the dataset with selected categories
categories = ['talk.politics.misc', 'sci.med']
newsgroups_data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

In [4]:
import re
from collections import Counter
from typing import List, Set
import heapq

def preprocess_text(text: str) -> List[str]:
    """
    Tokenize and clean text by:
    1. Converting to lowercase
    2. Removing punctuation
    3. Splitting on whitespace
    4. Removing empty tokens
    """
    #
    # % -- Your Implementation -- %
    #
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.split()
    return text

# Get first 1000 documents
documents = newsgroups_data.data[:1000]

# Tokenize all documents
tokenized_docs = [preprocess_text(doc) for doc in documents]

# Now write a fucntion to get the vocabulary
def get_vocab(tokenized_docs: List[List[str]], vocab_size: int) -> Set[str]:
    """
    Create vocabulary from tokenized documents by:
    1. Counting word frequencies across all documents
    2. Taking the `vocab_size` most common words

    Args:
        tokenized_docs: List of documents, where each document is a list of tokens

    Returns:
        Set of vocabulary words
    """
    # storing tuples (frequency, word)
    freqs = {}
    for doc in tokenized_docs:
        for word in doc:
            if word not in freqs:   
                freqs[word] = 0
            freqs[word] += 1
    sorted_freqs = sorted(freqs.items(), key=lambda x: x[1], reverse=True)
    return set([word for word, freq in sorted_freqs[:vocab_size]])  



vocab = get_vocab(tokenized_docs, vocab_size=5000)

print(f"Vocabulary size: {len(vocab)}")
print("\nSample vocabulary words:")
print(list(vocab)[:10])


Vocabulary size: 5000

Sample vocabulary words:
['tend', 'tour', 'friends', 'yeah', 'abortion', 'dynamics', 'minute', 'mass', 'cornea', 'game']


## 2. Computing the co-occurrence matrix (10 points)

Assume co-occurrence means if a word appears in the same document and within a window of size 10 of another word. Compute the co-occurrence matrix based on raw frequencies.

For example in the sentence: "the hungry cat ate the small fish near the blue lake"

With window size 4, the co-occurrences for the word "cat":

- Left context (within 2 words): "the", "hungry"

- Right context (within 1 words): "ate"

- So "cat" co-occurs once each with: the, hungry, ate

Words outside the window like "fish", "near", "blue", "lake" are not counted as co-occurrences with "cat" in this case.

Complete the following steps:

- Construct a co-occurrence matrix. This matrix will have dimensions `vocab_size` x `vocab_size` (5000 x 5000 in our case).
- For each document, iterate through its words. For each word, increment the corresponding entries in the co-occurrence matrix for its neighboring words within a specified window size (e.g., a window of 5 words to the left and right).


In [5]:
import numpy as np

def compute_cooccurrence_matrix(tokenized_docs: List[List[str]], vocab: Set[str], window_size: int) -> np.ndarray:
    """
    Compute word co-occurrence matrix from tokenized documents using a sliding window approach.

    Args:
        tokenized_docs (List[List[str]]): List of documents where each document is a list of tokens
        vocab (Set[str]): Set of vocabulary words to consider
        window_size (int): Number of words to consider
            if odd, there are window_size//2 words on each side of the center word
            if even, there are window_size//2 words on the left and window_size//2 - 1 words on the right

    Returns:
        np.ndarray: Co-occurrence matrix of shape (vocab_size, vocab_size) where entry [i,j]
                   represents how many times word i appears within the window of word j
    """
    #
    # % -- Your Implementation -- %
    #
    word_to_idx = {word: i for i, word in enumerate(sorted(vocab))}
    vocab_size = len(vocab)
    
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size))
    
    for doc in tokenized_docs:
        for i, word in enumerate(doc):
            if word not in vocab:
                continue
            
            if window_size % 2 == 0:
                left = i - window_size // 2
                right = i + window_size // 2 
            else:  # odd
                left = i - window_size // 2
                right = i + window_size // 2 + 1
            
            for j in range(max(0, left), min(len(doc), right)): # ensure that we are not out of bounds
                if i != j and doc[j] in vocab:
                    cooccurrence_matrix[word_to_idx[word], word_to_idx[doc[j]]] += 1
    
    return cooccurrence_matrix


cooccurrence_matrix = compute_cooccurrence_matrix(tokenized_docs, vocab, window_size=10)
print(f"Co-occurrence matrix shape: {cooccurrence_matrix.shape}")

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    #
    # % -- Your Implementation -- %
    #
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


Co-occurrence matrix shape: (5000, 5000)


## 3. Application: Word Similarity (10 points)

- Implement a function to calculate the cosine similarity between two *word* vectors.
- For the following words, find the 5 most similar words to each of your chosen words based on their raw co-occurrence representations using the cosine similarity function.
[concerned, lone, matthew]
- Print the chosen words and their 5 most similar words, along with their similarity scores.

Recall that `Cosine Similarity(A, B) = (A . B) / (||A|| * ||B||)`
where `.` denotes the dot product, and `||A||` is the magnitude (Euclidean norm) of vector A.

Note: You may pass `vocab` into `find_most_similar_words()` if desired.


In [6]:
words = ['concerned', 'lone', 'matthew']

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    #
    # % -- Your Implementation -- %
    #
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def find_most_similar_words(words: List[str], cooccurrence_matrix: np.ndarray) -> List[Tuple[str, List[Tuple[str, float]]]]:
    """
    Find the 5 most similar words to each input word based on their co-occurrence representations.

    Args:
        words (List[str]): List of words to find similar words for
        cooccurrence_matrix (np.ndarray)

    Returns:
        List[Tuple[str, List[Tuple[str, float]]]]: For each input word, returns a tuple containing:
            - The input word
            - A list of 5 tuples, each containing:
                - A similar word
                - The cosine similarity score between the input and similar word

    Example:
        >>> result = find_most_similar_words(['cat'], cooccurrence_matrix)
        >>> result
        [('cat', [('dog', 0.85), ('kitten', 0.76), ('pet', 0.72), ...])]
    """
    #
    # % -- Your Implementation -- %
    #
    
    word_to_idx = {word: i for i, word in enumerate(sorted(vocab))}
    idx_to_word = {i: word for word, i in word_to_idx.items()}
    
    results = []
    
    for word in words:
        if word not in vocab:
            results.append((word, []))
            continue
        
        word_idx = word_to_idx[word]
        word_vec = cooccurrence_matrix[word_idx]
        
        similarities = []
        for i in range(len(vocab)):
            if i == word_idx:
                continue
            other_vec = cooccurrence_matrix[i]
            sim = cosine_similarity(word_vec, other_vec)
            similarities.append((sim,idx_to_word[i]))
        
        similarities.sort(reverse=True)
        top_5 = similarities[:5]
        for tup in top_5:
            results.append((tup[1],tup[0]))
            
    return results

print(find_most_similar_words(words, cooccurrence_matrix))

[('being', 0.7846449225081631), ('what', 0.7793101401464834), ('this', 0.7709015495877185), ('that', 0.7676644685283173), ('as', 0.7665753424810565), ('scientist', 0.6320018010533753), ('few', 0.6301430726198638), ('physician', 0.623335714251067), ('bit', 0.6146013164009455), ('chance', 0.5840981798624632), ('deane', 0.6555213366563067), ('deanebinahccbrandeisedu', 0.6313641498019763), ('david', 0.3519381814343646), ('wright', 0.29777500019127906), ('dn', 0.2440231932656964)]


## 4. TF-IDF weighting (10 points)

In this part you will implement TF-IDF weighting, which is a numerical statistic that reflects how important a word is to a document in a collection or corpus. TF-IDF consists of two components:
1. Term Frequency (TF): How frequently a word appears in a document  
2. Inverse Document Frequency (IDF): How unique or rare the word is across all documents  

Assume we have a term-document matrix of co-occurrences.
Then we can use TF-IDF to reweight the raw frequences.

First compute the TF. Recall that TF is:

tf(t,d) = 1 + log(count(t,d)) if term t appears in document d  
tf(t,d) = 0 if term t doesn't appear in document d

In [7]:
def compute_tf(document: Union[str, List[str]], vocab: Set[str]) -> Dict[str, float]:
    """
    Compute term frequencies in a document.

    """
    #
    # % -- Your Implementation -- %
    #
    word_counts = {}
    for word in document:
        if word in vocab:
            word_counts[word] = word_counts.get(word, 0) + 1
    
    # Compute TF for each word in vocab
    tf_scores = {}
    for word in vocab:
        if word in word_counts:
            tf_scores[word] = 1 + np.log(word_counts[word])
        else:
            tf_scores[word] = 0.0
    
    return tf_scores

### Compute inverse document frequency
Implement a function to calculate document frequencies across a corpus.
Recall that inverse document frequency is:

log ( N / df ), where:

- **N** is the total number of documents in the corpus.
- **df** is the document frequency, i.e., the number of documents in which the term appears.

In [8]:
def compute_idf(documents: Union[List[str], List[List[str]]], vocab: Set[str]) -> Dict[str, float]:
    """
    Compute inverse document frequency for all terms in the corpus.
    Returns a dictionary mapping terms to their IDF scores.
    """
    #
    # % -- Your Implementation -- %
    #
    N = len(documents)
    
    doc_freq = {}
    for word in vocab:
        doc_freq[word] = 0
    
    for doc in documents:
        unique_words = set(doc)
        for word in unique_words:
            if word in vocab:
                doc_freq[word] += 1
    
    idf_scores = {}
    for word in vocab:
        if doc_freq[word] > 0:
            idf_scores[word] = np.log(N / doc_freq[word])
    return idf_scores
    

### Caclculate the co-occurrence matrix with TF-IDF weights.

Now we want to calculate TF-IDF scores for each term in each document. We'll combine the term frequency (TF) and inverse document frequency (IDF) scores we computed earlier to get the final TF-IDF weights.  
This will give us a measure of how important each word is to a document in the context of the entire corpus.

Recall that the TF-IDF score is calculated as:  
`tf-idf = tf * idf`  

Where:  
- tf is the term frequency (number of times term appears in document)  
- idf is the inverse document frequency `(log(N/df))`


In [9]:
def compute_tfidf(tokenized_docs: List[List[str]], vocab: Set[str]) -> np.ndarray:
    """
    Compute TF-IDF (Term Frequency-Inverse Document Frequency) matrix for a collection of documents.

    The TF-IDF score is calculated as:
    tf-idf(t,d) = tf(t,d) * idf(t)

    Args:
        tokenized_docs (List[List[str]]): List of documents, where each document is a list of tokens
        vocab (Set[str]): Set of vocabulary words to consider

    Returns:
        np.ndarray: TF-IDF matrix of shape (vocab_size, num_documents) where:
            - Each row represents a term from the vocabulary
            - Each column represents a document
            - Entry [i,j] represents the TF-IDF score of term i in document j

    Note:
        - IDF calculation includes +1 smoothing to avoid division by zero
        - TF calculation uses log normalization: 1 + log(term_count)
    """
    #
    # % -- Your Implementation -- %
    #
    vocab_size = len(vocab)
    num_docs = len(tokenized_docs)
    
    word_to_idx = {word: i for i, word in enumerate(sorted(vocab))}
    
    idf_scores = compute_idf(tokenized_docs, vocab)
    
    tfidf_matrix = np.zeros((vocab_size, num_docs))
    
    for doc_idx, doc in enumerate(tokenized_docs):
        tf_scores = compute_tf(doc, vocab)
        
        for word in vocab:
            word_idx = word_to_idx[word]
            tfidf_matrix[word_idx, doc_idx] = tf_scores[word] * idf_scores[word]
    
    return tfidf_matrix
word_document_matrix = compute_tfidf(tokenized_docs, vocab)

### Application in search (10 points)

Now we will use TF-IDF representations to implement a simple search function.
In this function we match the most similar documents to a given query.
The query itself should be converted to a TF-IDF vector as above.

Implement the following function.

In [11]:
def search_documents(query: str, tokenized_docs: List[List[str]], tfidf_matrix: np.ndarray, vocab: Set[str]) -> List[Tuple[int, float]]:
    """
    Search for documents most relevant to a query using TF-IDF representations.

    Args:
        query (str): Search query string
        tokenized_docs (List[List[str]]): List of documents where each document is a list of tokens
        tfidf_matrix (np.ndarray): TF-IDF matrix of shape (vocab_size, num_documents)
        vocab (Set[str]): Vocabulary set used for TF-IDF computation

    Returns:
        List[Tuple[int, float]]: List of (document_index, similarity_score) tuples,
            sorted by relevance (highest similarity first)
    """
    #
    # % -- Your Implementation -- %
    #
    word_to_idx = {word: i for i, word in enumerate(sorted(vocab))}
    query_tokens = preprocess_text(query)
    query_tf = compute_tf(query_tokens, vocab)
    idf_scores = compute_idf(tokenized_docs, vocab)

    query_vec = np.zeros(len(vocab))
    for word in vocab:
        query_vec[word_to_idx[word]] = query_tf[word] * idf_scores.get(word, 0)

    query_norm = np.linalg.norm(query_vec)
    results = []
    for doc_idx in range(tfidf_matrix.shape[1]):
        doc_vec = tfidf_matrix[:, doc_idx]
        denom = query_norm * np.linalg.norm(doc_vec)
        if denom > 0:
            score = float(np.dot(query_vec, doc_vec) / denom)
        else:
            score = 0
        results.append((doc_idx, score))

    results.sort(key=lambda x: x[1], reverse=True)
    return results

# Example usage:
query = "medical treatment for heart disease"
results = search_documents(query, tokenized_docs, word_document_matrix, vocab)

print(f"Top 5 most relevant documents for query: '{query}'\n")
for doc_idx, score in results[:5]:
    # Print first 100 characters of each document
    preview = ' '.join(tokenized_docs[doc_idx])[:100] + "..."
    print(f"Document {doc_idx} (score: {score:.3f}):")
    print(f"{preview}\n")

Top 5 most relevant documents for query: 'medical treatment for heart disease'

Document 843 (score: 0.228):
herman i would think you of all people wouldcould distinguish between health and treatment of diseas...

Document 894 (score: 0.191):
turkish president turgur ozal has passed away today after a heart attack in ankara at 1100 am gmt mr...

Document 73 (score: 0.189):
stimulation of the vagus nerve slows the heart and drops the blood pressure gordon banks n3jxp skept...

Document 44 (score: 0.185):
the term arrhythmia is usually used to encompass a wide range of abnormal heart rhythms cardiac dysr...

Document 441 (score: 0.157):
the funny thing is the personaly stories about reactions to msg vary so greatly some said that their...



## 5. Reflection questions (10 points)

Now please answer the following reflection questions:

1- How do the most similar words differ when using raw co-occurrence counts versus TF-IDF weighted representations? What might explain these differences?

# Raw co-occurance rewards frequency and local proximity where TF-IDF brings out the words that are distinctive to fewer documents. Thus Raw co-occurnace will be more likley to return many frequent and generic neighbors where TF-IDF is better at returning more content related terms 

2. What are the limitations of using a fixed window size for co-occurrence counting? How might this impact the quality of word similarities for different types of words (e.g., nouns vs. verbs)?

# A fixed window size may not be optimal because if it is too small we can miss out on long range semantics but if it is too large it is xlumsy and can add noise and unrelated words. Nouns may benefit from nearby descriptive modifiers where verbs will depend on more long range structure, so a fixed window may weigh these classes in a biased manner. 

3- How does the choice of preprocessing steps (like removing punctuation, converting to lowercase, etc.) affect the resulting word representations? Can you think of cases where preserving some of this information might be beneficial?

# Yes, for example Apple vs apple mean a technology company versus a food, or U.S vs us, or even removing ! and ? remove some semantics from sentances that signifigantly alter meaning. 